In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import functools
import re

from cryptic.evaluation.evaluations import GeorgeHoQA
from cryptic.models.oai import OpenAIQA
from cryptic.models.validators import SubWordConsistencyValidator
from cryptic.prompts.interface import PromptInterface

In [ ]:
CSV_FILENAME = "data/examples/qc_1711.csv"
DUMMY_CLUE = "A farmer's son"
NUM_CLUES = 2

Check that injection of clue and num letters into prompt template is working correctly:

In [ ]:
prompt_interface = PromptInterface("cryptic.prompts.cot.ben_v1")
prompt = prompt_interface.inject_prompt(DUMMY_CLUE, 5)
print(prompt)

Check extraction of answer is working correctly

In [ ]:
out = prompt_interface.prompt_templates["example_output"]

In [ ]:
res = prompt_interface.extract_answer(out)
res

In [ ]:
prompt_interface.decompose(res["wordplay"])

In [ ]:
validator = SubWordConsistencyValidator(prompt_interface)

In [ ]:
evaluation = GeorgeHoQA(CSV_FILENAME, num_answers=2, num_clues=NUM_CLUES)

In [ ]:
model = OpenAIQA(
    prompt_interface=prompt_interface,
    model_name="text-davinci-003",
    max_tokens=64,
    validators=[validator],
    validator_names=["consistent_decomposition"]
)

In [ ]:
df = evaluation.qa_frame.df
df.head(3)

First feed a single clue through the API to look at model output and check that we are correctly parsing responses.

This might need to be tweaked: e.g. removal of newline / space chars etc,
to convert into a standard answer format

In [ ]:
clue, num_letters = evaluation.qa_frame.sample(1).df.iloc[0][["clue", "num_letters"]]

In [ ]:
response = model.get_response(clue, num_letters, num_answers=2)
response

In [ ]:
out = response["choices"][0]["text"]

In [ ]:
res = prompt_interface.extract_answer(response["choices"][0]["text"])
res

In [ ]:
prompt_interface.decompose(res["wordplay"])

In [ ]:
validator.validate(res["answer"], res["predicted_definition"], res["wordplay"])

Once we're confident that answers are being extracted correctly we can run a whole set of clues through

In [ ]:
metrics, answer_df = evaluation.run(model)

In [ ]:
answer_df

In [ ]:
import pandas as pd
pd.options.display.max_colwidth = 200

In [ ]:
answer_df[answer_df["correct"]][["clue", "answer", "prediction_0", "prediction_1", "explanation_0", "explanation_1"]]